In [1]:
# Dependencies installation for technical indicator testing framework
# backtesting: Framework for strategy testing and visualization  
# yfinance: Yahoo Finance data acquisition library
# ipywidgets: Interactive widgets for Jupyter notebooks
# ta: Technical analysis indicators library
!pip install backtesting yfinance ipywidgets ta --quiet

In [2]:
# ===================================================================
# TECHNICAL INDICATOR TESTING FRAMEWORK
# ===================================================================

# Install missing package
%pip install ta

# Import necessary libraries for data handling and backtesting
import yfinance as yf
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import ipywidgets as widgets
from IPython.display import display, clear_output
import ta

# -------------------------------------------------------------------
# STRATEGY DEFINITIONS
# -------------------------------------------------------------------

class SMAStrategy(Strategy):
    """
    Simple Moving Average Crossover Strategy
    - Generates buy signal when fast MA crosses above slow MA
    - Generates sell signal when slow MA crosses above fast MA
    """
    
    # Strategy parameters: moving average periods
    n1 = 20  # Short-term moving average window
    n2 = 50  # Long-term moving average window

    def init(self):
        """Initialize strategy indicators"""
        close = pd.Series(self.data.Close)
        
        # Fast moving average (20-period)
        self.sma1 = self.I(lambda x: x.rolling(self.n1).mean(), close)
        
        # Slow moving average (50-period)  
        self.sma2 = self.I(lambda x: x.rolling(self.n2).mean(), close)

    def next(self):
        """Execute trading logic for each time step"""
        # Long entry: fast MA crosses above slow MA (bullish signal)
        if crossover(self.sma1, self.sma2):
            self.buy(size=10)
            
        # Short entry: slow MA crosses above fast MA (bearish signal)
        elif crossover(self.sma2, self.sma1):
            self.sell(size=10)

class RSIStrategy(Strategy):
    """
    RSI (Relative Strength Index) Strategy
    - Generates buy signal when RSI crosses above oversold level (30)
    - Generates sell signal when RSI crosses below overbought level (70)
    """
    
    # RSI parameters
    rsi_period = 14
    oversold = 30
    overbought = 70

    def init(self):
        """Initialize RSI indicator"""
        close = pd.Series(self.data.Close)
        
        # Calculate RSI using ta library
        def calculate_rsi(prices):
            return ta.momentum.RSIIndicator(prices, window=self.rsi_period).rsi()
        
        self.rsi = self.I(calculate_rsi, close)

    def next(self):
        """Execute RSI-based trading logic"""
        current_rsi = self.rsi[-1]
        
        # Buy when RSI crosses above oversold level
        if current_rsi > self.oversold and self.rsi[-2] <= self.oversold:
            if not self.position:
                self.buy(size=10)
        
        # Sell when RSI crosses below overbought level
        elif current_rsi < self.overbought and self.rsi[-2] >= self.overbought:
            if self.position:
                self.position.close()

# -------------------------------------------------------------------
# TECHNICAL INDICATOR TESTING FRAMEWORK CLASS
# -------------------------------------------------------------------

class TechnicalIndicatorFramework:
    """
    Framework for testing multiple technical indicator strategies
    """
    
    def __init__(self):
        self.data = None
        self.results = {}
        
        # Available strategies
        self.strategies = {
            'SMA Crossover': SMAStrategy,
            'RSI Strategy': RSIStrategy
        }
    
    def fetch_data(self, ticker, start_date, end_date):
        """Fetch and process market data"""
        try:
            # Download data from Yahoo Finance
            data = yf.download(ticker, start=start_date, end=end_date, group_by='ticker')
            
            # Process multi-level column structure
            if hasattr(data.columns, 'nlevels') and data.columns.nlevels > 1:
                data.columns = data.columns.map(lambda x: x[1] if isinstance(x, tuple) else x)
            
            # Select essential OHLCV columns
            self.data = data[["Open", "High", "Low", "Close", "Volume"]]
            
            print(f"✅ Data fetched successfully for {ticker}")
            print(f"📊 Date range: {start_date} to {end_date}")
            print(f"📈 Total trading days: {len(self.data)}")
            
            return True
            
        except Exception as e:
            print(f"❌ Error fetching data: {str(e)}")
            return False
    
    def run_backtest(self, strategy_name):
        """Run backtest for specified strategy"""
        if self.data is None:
            print("❌ No data available. Please fetch data first.")
            return None
        
        try:
            strategy_class = self.strategies[strategy_name]
            
            # Initialize and run backtest
            bt = Backtest(self.data, strategy_class, cash=10_000, commission=0.002)
            results = bt.run()
            
            # Store results
            self.results[strategy_name] = {
                'backtest': bt,
                'results': results
            }
            
            print(f"✅ {strategy_name} backtest completed successfully")
            return results
            
        except Exception as e:
            print(f"❌ Error running {strategy_name} backtest: {str(e)}")
            return None
    
    def display_results(self, strategy_name):
        """Display backtest results and visualization"""
        if strategy_name in self.results:
            result_data = self.results[strategy_name]
            
            print(f"\n{'='*60}")
            print(f"{strategy_name.upper()} BACKTEST RESULTS")
            print(f"{'='*60}")
            
            # Display key metrics
            results = result_data['results']
            key_metrics = {
                'Total Return': f"{results['Return [%]']:.2f}%",
                'Sharpe Ratio': f"{results['Sharpe Ratio']:.3f}",
                'Maximum Drawdown': f"{results['Max. Drawdown [%]']:.2f}%",
                'Win Rate': f"{results['Win Rate [%]']:.1f}%",
                'Total Trades': results['# Trades']
            }
            
            for metric, value in key_metrics.items():
                print(f"{metric:20}: {value}")
            
            print(f"{'='*60}")
            
            # Generate plot
            result_data['backtest'].plot()
            
        else:
            print(f"❌ No results found for {strategy_name}")

# -------------------------------------------------------------------
# INTERACTIVE FRAMEWORK INTERFACE
# -------------------------------------------------------------------

# Initialize the framework
framework = TechnicalIndicatorFramework()

# Create interactive widgets
ticker_widget = widgets.Text(
    value='AAPL',
    placeholder='Enter ticker symbol (e.g., AAPL, MSFT, GOOGL)',
    description='Ticker:',
    style={'description_width': 'initial'}
)

start_date_widget = widgets.DatePicker(
    description='Start Date:',
    value=pd.to_datetime('2020-01-01').date(),
    style={'description_width': 'initial'}
)

end_date_widget = widgets.DatePicker(
    description='End Date:',
    value=pd.to_datetime('2023-01-01').date(),
    style={'description_width': 'initial'}
)

strategy_widget = widgets.Dropdown(
    options=['SMA Crossover', 'RSI Strategy'],
    value='SMA Crossover',
    description='Strategy:',
    style={'description_width': 'initial'}
)

# Output widget for results
output_widget = widgets.Output()

def run_backtest_button_clicked(b):
    """Handle backtest button click"""
    with output_widget:
        clear_output()
        
        # Get parameters
        ticker = ticker_widget.value.upper()
        start_date = start_date_widget.value.strftime('%Y-%m-%d')
        end_date = end_date_widget.value.strftime('%Y-%m-%d')
        strategy = strategy_widget.value
        
        print(f"🚀 Starting backtest with parameters:")
        print(f"   Ticker: {ticker}")
        print(f"   Start Date: {start_date}")
        print(f"   End Date: {end_date}")
        print(f"   Strategy: {strategy}")
        print("\n" + "-"*50)
        
        # Fetch data and run backtest
        if framework.fetch_data(ticker, start_date, end_date):
            results = framework.run_backtest(strategy)
            if results is not None:
                framework.display_results(strategy)

# Create run button
run_button = widgets.Button(
    description='🚀 Run Backtest',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)
run_button.on_click(run_backtest_button_clicked)

# Display the interface
print("📊 TECHNICAL INDICATOR TESTING FRAMEWORK")
print("="*50)
print("Configure your backtest parameters and click 'Run Backtest'")

display(widgets.VBox([
    widgets.HBox([ticker_widget, strategy_widget]),
    widgets.HBox([start_date_widget, end_date_widget]),
    run_button,
    output_widget
]))

Note: you may need to restart the kernel to use updated packages.


/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

📊 TECHNICAL INDICATOR TESTING FRAMEWORK
Configure your backtest parameters and click 'Run Backtest'
